In [ ]:
import pandas as pd
import warnings
import numpy as np
import re
import math
import firebase_admin
from firebase_admin import credentials, firestore, auth
from collections import defaultdict

warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("data.csv")
df


In [ ]:
# Mapping from 2025 CSV columns to internal field names (aligned with 2024 schema)

column_mapping_2025 = {
    # --- Core identity ---
    "Attending SLBTS as": "studentGroup",
    "A.Student Name": "studentFullName",
    "A.Student Date of Birth": "dateOfBirth",
    "A.Student Gender": "gender",
    "A.District": "district",
    "A.Name of the Samithi": "samithiName",
    "A.Student - Year of Joining Balvikas": "yearOfJoiningBalvikas",

    # --- Events ---
    "A.Select First Competition": "event1",
    "A.Select Second Competition": "event2",

    # --- Travel & transport ---
    "Date of Arrival": "arrivalDate",
    "Mode of Travel - Arrival": "modeOfTravel",
    "Pickup Facility is required?": "needsPickup",
    "Pickup Point": "pickupPoint",
    "Date of Departure": "departureDate",
    "Mode of Travel - Departure": "modeOfTravelForDrop",
    "Drop Facility is Required?": "needsDrop",
    "Drop Off Point": "dropOffPoint",

    # --- Accompanying & accommodation ---
    "Will anyone be accompanying? ": "hasAccompanyingAdults",
    "No of Adults & Kids Accompanying [Gents - Adult]": "numMaleAccompanying",
    "No of Adults & Kids Accompanying [Mahilas - Adult]": "numFemaleAccompanying",
    "No of Adults & Kids Accompanying [Boy - Kid]": "numBoyAccompanyingKid",   # NEW
    "No of Adults & Kids Accompanying [Girl - Kid]": "numGirlAccompanyingKid", # NEW
    "Total Accompany": "totalAccompanyingCount",                               # NEW

    "Name of the Adult In-charge": "accompanyingPersonName",
    "Relationship with the Registered Participant ": "accompanyingPersonRelation",
    "Phone no. of the Adult In-charge": "accompanyingPersonContact",
    "Email of the Adult In-charge": "accompanyingPersonEmail",                 # NEW

    # Two accommodation questions – keep both so we don't lose information
    "Accommodation is required? ": "needsAccommodation",
    "Accommodation is required": "accommodationForInchargeOrGroup",           # NEW

    # Duplicate district at end – for adult in-charge / admin context
    "A.District.1": "inchargeDistrict",                                       # NEW

    # --- Other meta/contact fields ---
    "Email address": "formFillerEmail",                                       # NEW
    "A.Student/ Guru/ Parent - Email ID": "contactEmail",                     # NEW
    "A.Student/ Guru/ Parent - Phone No.": "contactPhone",                    # NEW

    # Return journey food
    "Return Journey Food Packet: Required? ": "needsReturnFoodPacket",        # NEW
}

# Codes reused from 2024 logic

district = [
    "Chennai South",
    "Coimbatore",
    "Tiruvannamalai",
    "Tirunelveli",
    "Kanchipuram North",
    "Thanjavur",
    "Chennai South East",
    "Kanchipuram South",
    "Madurai",
    "Virudhunagar",
    "Trichy",
    "Dharmapuri / Krishnagiri",
    "Cuddalore",
    "Kanyakumari",
    "Chennai West",
    "Salem",
    "Nilgiris",
    "Tiruvallur East",
    "Tuticorin",
    "Tirupur",
    "Chennai North",
    "Sivaganga&Ramnad",
    "Erode",
    "Chennai North West",
    "Puducherry",
    "Vellore",
    "Dindigul",
    "Chennai East Coast",
    "Karur",
    "Theni",
    "Mayiladuthurai",
    "Nagapattinam",
    "Villupuram",
]

# Map district name to 2‑digit code (same ordering as 2024)

district_code = {name: str(i + 1).zfill(2) for i, name in enumerate(district)}

group_code = {
    "Group 1": "G1",
    "Group 2": "G2",
    "Group 3": "G3",
    "Group 4": "G4",              # treat Group 4 same as General Category
    "General Category": "G4",
}

gender_code = {"Male": "M", "Female": "F"}


def processData_2025(df_raw: pd.DataFrame) -> pd.DataFrame:
    """Sort by Timestamp, drop it, and rename 2025 columns to internal names."""
    df = df_raw.copy()

    # Ensure Timestamp is parsed and used for stable ordering
    if "Timestamp" in df.columns:
        df["Timestamp"] = pd.to_datetime(df["Timestamp"], dayfirst=True, errors="coerce")
        df = df.sort_values("Timestamp").reset_index(drop=True)
        df = df.drop(columns=["Timestamp"])

    # Rename according to 2025 mapping (ignore columns that are not in the dict)
    df = df.rename(columns=column_mapping_2025)
    return df


def _build_registered_events(record: dict) -> list:
    """Build the registeredEvents list from event1/event2, adding group prefixes.

    Quiz is inferred from event text containing the word "quiz".
    """
    events = []
    quiz = False

    group = str(record.get("studentGroup", "")).strip()
    prefix = group_code.get(group, "")

    for key in ("event1", "event2"):
        val = str(record.get(key, "")).strip()
        if not val or val.lower().startswith("no second competition"):
            continue

        # Infer Quiz separately
        if "quiz" in val.lower():
            quiz = True
            continue

        # Add group prefix if we know it and it's not already present
        if prefix and not val.startswith(prefix + "-") and "GROUP" not in val.upper():
            val = f"{prefix}-{val}"

        if "DRAWING" not in val.upper() and "QUIZ" not in val.upper() and "G1" not in val.upper() and "ELOCUTION" not in val.upper():
            if record.get("gender", "") == "Male":
                val = f"{val}-Boys"
            else:
                val = f"{val}-Girls"
        
        val = val.replace(" ", "")
            
        events.append(val)

    if quiz:
        events.append(f"{prefix}-Quiz")

    # Drop empties if any
    return [e for e in events if str(e).strip() != ""]


def prepareData_2025(df_raw: pd.DataFrame) -> list[dict]:
    """Transform raw 2025 form CSV into a list of records ready for Firestore upload.

    The output schema matches 2024 (studentId, registeredEvents, counts, etc.)
    with additional 2025‑specific fields preserved.
    """
    df = processData_2025(df_raw)
    records = df.to_dict(orient="records")

    uid = 1
    out = []

    for rec in records:
        # Normalise NaN to empty string for string fields
        for k, v in list(rec.items()):
            if isinstance(v, float) and math.isnan(v):
                rec[k] = ""

        # Ensure legacy keys exist so downstream code can rely on them
        string_defaults = [
            "dateOfJoiningBalvikas",
            "hasPassedGroup2Exam",
            "arrivalTime",
            "departureTime",
            "accompanyingPersonGender",
            "foodAllergies",
            "checkInDate",
            "checkInTime",
            "checkOutDate",
            "checkOutTime",
            "overallRegistrationStatus",
            "remarks",
        ]
        for key in string_defaults:
            rec.setdefault(key, "")

        numeric_defaults_zero = [
            "numNonParticipatingSiblings",
            "numMaleAccompanyingNeedAccommodation",
            "numFemaleAccompanyingNeedAccommodation",
            "totalParticipatingEvents",
        ]
        for key in numeric_defaults_zero:
            rec.setdefault(key, 0)

        # Coerce accompanying counts and derive numNonParticipatingSiblings
        for key in ["numMaleAccompanying", "numFemaleAccompanying",
                    "numBoyAccompanyingKid", "numGirlAccompanyingKid",
                    "totalAccompanyingCount"]:
            if key in rec:
                val = rec[key]
                if isinstance(val, str) and val.strip() == "":
                    rec[key] = 0
                else:
                    try:
                        rec[key] = int(float(val))
                    except Exception:
                        rec[key] = 0

        rec["numNonParticipatingSiblings"] = (
            rec.get("numBoyAccompanyingKid", 0) + rec.get("numGirlAccompanyingKid", 0)
        )

        if "Devotional Singing" in rec.get("event1", ""):
            rec["event1"] = "GROUP-Devotional-Singing"
        if "Devotional Singing" in rec.get("event2", ""):
            rec["event2"] = "GROUP-Devotional-Singing"

        if "Altar" in rec.get("event1", ""):
            rec["event1"] = "GROUP-Altar-Decoration"
        if "Altar" in rec.get("event2", ""):
            rec["event2"] = "GROUP-Altar-Decoration"

        if "Namakam" in rec.get("event1", ""):
            rec["event1"] = "GROUP-Namakam-Chanting"
        if "Namakam" in rec.get("event2", ""):
            rec["event2"] = "GROUP-Namakam-Chanting"

        # Build registeredEvents from event1/event2
        registered_events = _build_registered_events(rec)
        rec["registeredEvents"] = registered_events
        rec["totalParticipatingEvents"] = len(registered_events)

        # Generate studentId using same pattern as 2024
        group = str(rec.get("studentGroup", "")).strip()
        gender = str(rec.get("gender", "")).strip()
        district_name = str(rec.get("district", "")).strip()

        group_part = group_code.get(group, "G0")
        gender_part = gender_code.get(gender, "U")
        district_part = district_code.get(district_name, "00")
        events_part = str(len(registered_events)).zfill(2)
        uid_part = str(uid).zfill(3)

        rec["studentId"] = f"{group_part}{gender_part}{district_part}{events_part}{uid_part}"
        uid += 1

        # Set a sensible default for overallRegistrationStatus if empty
        if not rec.get("overallRegistrationStatus"):
            rec["overallRegistrationStatus"] = "Accepted"

        out.append(rec)

    return out


# Example: process current df (loaded above) into final records
prepared_records_2025 = prepareData_2025(df)
prepared_df_2025 = pd.DataFrame(prepared_records_2025)
prepared_df_2025.head()



In [ ]:
prepared_df_2025.iloc[0]

In [ ]:
# Initialize Firebase Admin SDK
# Using credentials from parent directory
cred = credentials.Certificate("../slts-8e29d-firebase-adminsdk-i6d1u-d3ea7654ac.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

print("Firebase initialized successfully")


In [ ]:
def delete_collection_v2(coll_ref, batch_size=10):
    """Delete all documents in the registrationData_v2 collection.
    
    Args:
        coll_ref: Firestore collection reference
        batch_size: Number of documents to delete per batch
    """
    if batch_size == 0:
        return
    
    docs = coll_ref.list_documents(page_size=batch_size)
    deleted = 0
    
    for doc in docs:
        doc.delete()
        deleted += 1
    
    if deleted >= batch_size:
        return delete_collection_v2(coll_ref, batch_size)
    
    print(f"Deleted {deleted} document(s) from collection")


def addDataToFirebase_v2(prepared_records):
    """Upload prepared 2025 registration data to Firestore collection registrationData_v2.
    
    Args:
        prepared_records: List of dictionaries ready for Firestore upload
    """
    collection_name = "registrationData_v2"
    collection_ref = db.collection(collection_name)
    
    total = len(prepared_records)
    for i, record in enumerate(prepared_records, 1):
        student_id = record.get("studentId", f"unknown_{i}")
        collection_ref.document(student_id).set(record)
        if i % 50 == 0 or i == total:
            print(f"Uploaded {i}/{total} records...")
    
    print(f"Successfully uploaded {total} records to {collection_name}")


In [ ]:
# Step 1: Delete existing data in registrationData_v2 collection
collection_name_v2 = "registrationData_v2"
collection_ref_v2 = db.collection(collection_name_v2)

print(f"Deleting existing data from {collection_name_v2}...")
delete_collection_v2(collection_ref_v2, batch_size=10)

# Step 2: Upload prepared 2025 data to registrationData_v2
print(f"\nUploading 2025 data to {collection_name_v2}...")
addDataToFirebase_v2(prepared_records_2025)

print("\n✅ All done! Data uploaded to Firebase successfully.")


In [ ]:
prepared_df_2025['gender'].unique()

In [ ]:
# Event evaluation criteria mapping based on 2025 PDF circular
# This maps event names to their evaluation rubrics

def get_event_eval_criteria(event_name: str) -> dict:
    """
    Returns evaluation criteria for an event based on the 2025 PDF circular.
    
    Args:
        event_name: Event name in format like 'G1-Bhajans', 'G2-Bhajans-Boys', etc.
    
    Returns:
        Dictionary with evaluation criteria and marks
    """
    event_lower = event_name.lower()
    
    # Bhajans (50 marks) - All groups
    if "bhajans" in event_lower:
        return {
            "Shruthi": 10,
            "Bhaavam": 10,
            "Ragam": 10,
            "Taalam": 10,
            "Memory & Pronunciation": 10,
        }
    
    # Vedam Chanting (50 marks) - All groups
    if "vedam" in event_lower:
        return {
            "Pronunciation": 15,
            "Bhaavam": 5,
            "Intonation": 15,
            "Memory": 15,
        }
    
    # Elocution & Story Telling (30 marks) - Groups 1, 2, 3
    if "elocution" in event_lower or "storytelling" in event_lower or "story telling" in event_lower:
        return {
            "Presentation": 10,
            "Content": 10,
            "Language": 10,
        }
    
    # Drawing (30 marks) - All groups
    if "drawing" in event_lower:
        return {
            "Theme": 10,
            "Colour coordination": 10,
            "Layout": 10,
        }
    
    # Tamizh Chants (30 marks) - Groups 1, 2, 3
    if "tamizhchants" in event_lower or "tamizh chants" in event_lower:
        return {
            "Bhaavam": 5,
            "Tune": 5,
            "Pronunciation": 10,
            "Memory": 10,
        }
    
    # Slokas - Need to check PDF, but likely similar to Vedam or Tamizh Chants
    # Based on context, Slokas might follow similar pattern to Vedam
    if "slokas" in event_lower:
        return {
            "Bhaavam": 5,
            "Tune": 5,
            "Pronunciation": 10,
            "Memory": 10,
        }
    
    # Devotional Singing (60 marks) - Group Event
    if "devotional" in event_lower and "singing" in event_lower:
        return {
            "Shruthi": 10,
            "Bhaavam": 10,
            "Ragam": 10,
            "Taalam": 10,
            "Memory & Pronunciation": 10,
            "Harmony": 10,
        }
    
    # Rudram Namakam Chanting (50 marks) - Group Event
    if "namakam" in event_lower or "rudram" in event_lower:
        return {
            "Pronunciation": 15,
            "Bhaavam": 5,
            "Intonation": 15,
            "Memory": 15,
        }
    
    # Altar Decoration (30 marks) - Group Event
    if "altar" in event_lower and "decoration" in event_lower:
        return {
            "Aesthetics": 10,
            "Resource Management": 10,
            "Teamwork": 10,
        }
    
    # Quiz - No specific marks mentioned in PDF, using default
    if "quiz" in event_lower:
        return {
            "Knowledge": 10,
            "Understanding": 10,
            "Application": 10,
        }
    
    # Default fallback
    return {
        "Criteria 1": 10,
        "Criteria 2": 10,
    }


def get_event_groups(event_name: str) -> list:
    """
    Returns list of groups that can participate in this event.
    
    Args:
        event_name: Event name in format like 'G1-Bhajans', 'G2-Bhajans-Boys', etc.
    
    Returns:
        List of group names like ['Group 1'], ['Group 2', 'Group 3'], etc.
    """
    event_lower = event_name.lower()
    
    # Group 1 events
    if event_name.startswith("G1-"):
        return ["Group 1"]
    
    # Group 2 events
    if event_name.startswith("G2-"):
        return ["Group 2"]
    
    # Group 3 events
    if event_name.startswith("G3-"):
        return ["Group 3"]
    
    # Group events (Altar Decoration, Devotional Singing, Namakam)
    if event_name.startswith("GROUP-"):
        if "devotional" in event_lower:
            # Devotional Singing can have Group 1, 2, 3 (based on PDF context)
            return ["Group 1", "Group 2", "Group 3"]
        else:
            # Altar Decoration and Namakam are for Group 2 and 3
            return ["Group 2", "Group 3"]
    
    # Handle GROUP events with Boys/Girls suffix
    if "GROUP-" in event_name and ("-Boys" in event_name or "-Girls" in event_name):
        base_event = event_name.rsplit("-", 1)[0]  # Remove -Boys or -Girls
        base_lower = base_event.lower()
        if "devotional" in base_lower:
            return ["Group 1", "Group 2", "Group 3"]
        else:
            return ["Group 2", "Group 3"]
    
    # Quiz is for Group 3 and Group 4
    if "quiz" in event_lower:
        return ["Group 3", "Group 4"]
    
    return []


def get_event_code(event_name: str) -> str:
    """
    Generates event code (jRefId) similar to 2024 format.
    
    Args:
        event_name: Event name
    
    Returns:
        Event code like 'g1bh', 'g2bb', 'grdb', etc.
    """
    event_lower = event_name.lower().replace(" ", "").replace("-", "")
    
    # Group 1 events
    if event_name.startswith("G1-"):
        base = event_name[3:].lower().replace(" ", "").replace("-", "")
        if "bhajans" in base:
            return "g1bh"
        elif "slokas" in base:
            return "g1sl"
        elif "drawing" in base:
            return "g1dr"
        elif "storytelling" in base or "storytelling" in base:
            if "english" in base:
                return "g1se"
            elif "tamil" in base or "tamizh" in base:
                return "g1st"
        elif "tamizhchants" in base or "tamizhchants" in base:
            return "g1tc"
        elif "vedam" in base:
            return "g1vc"
        # Handle camelCase variations
        elif "story" in base and "telling" in base:
            if "english" in base:
                return "g1se"
            elif "tamil" in base or "tamizh" in base:
                return "g1st"
    
    # Group 2 events
    if event_name.startswith("G2-"):
        base = event_name[3:].lower().replace(" ", "").replace("-", "")
        if "bhajans" in base:
            if "boys" in base:
                return "g2bb"
            elif "girls" in base:
                return "g2bg"
        elif "slokas" in base:
            if "boys" in base:
                return "g2sb"
            elif "girls" in base:
                return "g2sg"
        elif "vedam" in base:
            if "boys" in base:
                return "g2vb"
            elif "girls" in base:
                return "g2vg"
        elif "elocution" in base:
            if "english" in base:
                return "g2ee"  # Same code, different gender
            elif "tamil" in base or "tamizh" in base:
                return "g2et"  # Same code, different gender
        elif "drawing" in base:
            return "g2dr"
        elif "tamizhchants" in base:
            if "boys" in base:
                return "g2tb"
            elif "girls" in base:
                return "g2tg"
    
    # Group 3 events
    if event_name.startswith("G3-"):
        base = event_name[3:].lower().replace(" ", "").replace("-", "")
        if "bhajans" in base:
            if "boys" in base:
                return "g3bb"
            elif "girls" in base:
                return "g3bg"
        elif "slokas" in base:
            if "boys" in base:
                return "g3sb"
            elif "girls" in base:
                return "g3sg"
        elif "vedam" in base:
            if "boys" in base:
                return "g3vb"
            elif "girls" in base:
                return "g3vg"
        elif "elocution" in base:
            if "english" in base:
                    return "g3ee"
            elif "tamil" in base or "tamizh" in base:
                    return "g3et"
        elif "drawing" in base:
            return "g3dr"
        elif "tamizhchants" in base:
            if "boys" in base:
                return "g3tb"
            elif "girls" in base:
                return "g3tg"
    
    # Group events - check for Boys/Girls variants first
    if event_name.startswith("GROUP-"):
        # Check for Boys/Girls suffix first
        has_boys = "boys" in event_lower
        has_girls = "girls" in event_lower
        
        if "devotional" in event_lower and "singing" in event_lower:
            if has_boys:
                return "grdb"
            elif has_girls:
                return "grdg"
            else:
                return "grd"  # Generic (will be expanded to Boys/Girls variants)
        elif "altar" in event_lower and "decoration" in event_lower:
            if has_boys:
                return "grab"
            elif has_girls:
                return "grag"
            else:
                return "gra"  # Generic (will be expanded to Boys/Girls variants)
        elif "namakam" in event_lower or "rudram" in event_lower:
            if has_boys:
                return "grrb"
            elif has_girls:
                return "grrg"
            else:
                return "grr"  # Generic (will be expanded to Boys/Girls variants)
    
    # Quiz
    if "quiz" in event_lower:
        return "grqz"
    
    # Default fallback
    return "unknown"


print("Event evaluation criteria functions defined successfully!")


In [ ]:
# Function to create judge account (same logic as 2024)
def addJudge(judgeName: str, eventName: str, db_instance=None, auth_instance=None):
    """
    Creates a judge account in Firebase Auth and adds judge info to Firestore.
    Same logic as 2024 codebase.
    Handles existing users gracefully by retrieving their UID and updating Firestore.
    
    Args:
        judgeName: Name/ID of the judge (e.g., "judge1g1bh")
        eventName: Name of the event this judge will judge
        db_instance: Firestore client instance (if None, will use global db)
        auth_instance: Firebase Auth instance (if None, will use global auth)
    """
    if db_instance is None:
        try:
            db_instance = db
        except NameError:
            raise ValueError("Firebase must be initialized first. Run the Firebase initialization cell.")
    
    if auth_instance is None:
        auth_instance = auth
    
    judgeEmail = f"{judgeName}@slts.cbe"
    judgePassword = f"{judgeName}@23111926"
    
    try:
        # Create Firebase Auth user
        judge = auth_instance.create_user(email=judgeEmail, password=judgePassword)
        judge_uid = judge.uid

        # Create userData document for judge
        user_ref = db_instance.collection("userData").document(judge_uid)
        user_ref.set({
            "email": judgeEmail,
            "name": judgeName,
            "id": judge_uid,
            "role": "judge",
            "event": eventName,
            "eventRef": db_instance.collection("eventData_v2").document(eventName),
        })

        # Update eventData_v2 document with judge info
        event_ref = db_instance.collection("eventData_v2").document(eventName)
        event_ref.update({
            "judgeIdList": firestore.ArrayUnion([judge_uid]),
            "judgeEmailList": firestore.ArrayUnion([judgeEmail]),
        })
        
        return judge_uid
    except Exception as e:
        error_str = str(e)
        # Handle case where user already exists
        if "EMAIL_EXISTS" in error_str or "already exists" in error_str.lower():
            try:
                # Get existing user by email
                existing_user = auth_instance.get_user_by_email(judgeEmail)
                judge_uid = existing_user.uid
                
                # Update userData document for judge (in case it doesn't exist or needs updating)
                user_ref = db_instance.collection("userData").document(judge_uid)
                user_ref.set({
                    "email": judgeEmail,
                    "name": judgeName,
                    "id": judge_uid,
                    "role": "judge",
                    "event": eventName,
                    "eventRef": db_instance.collection("eventData_v2").document(eventName),
                }, merge=True)  # Use merge=True to preserve existing fields

                # Update eventData_v2 document with judge info
                event_ref = db_instance.collection("eventData_v2").document(eventName)
                event_ref.update({
                    "judgeIdList": firestore.ArrayUnion([judge_uid]),
                    "judgeEmailList": firestore.ArrayUnion([judgeEmail]),
                })
                
                # Silently return the UID - user already exists, which is fine
                return judge_uid
            except Exception as lookup_error:
                print(f"  ⚠ Error looking up existing judge {judgeName} for {eventName}: {str(lookup_error)}")
                return None
        else:
            # For other errors, print warning and return None
            print(f"  ⚠ Error creating judge {judgeName} for {eventName}: {error_str}")
            return None


# Function to upload event metadata to Firestore
# Note: This requires Firebase to be initialized (run the Firebase init cell first)

def upload_event_metadata_to_firestore(event_list: list, db_instance=None, create_judges=True, num_judges_per_event=4):
    """
    Uploads event metadata with evaluation criteria to Firestore eventData_v2 collection.
    Optionally creates judge accounts for each event.
    
    Args:
        event_list: List of event names (dynamically generated from registration data)
        db_instance: Firestore client instance (if None, will use global db)
        create_judges: Whether to create judge accounts (default: True)
        num_judges_per_event: Number of judges to create per event (default: 4)
    """
    if db_instance is None:
        # Try to use global db if available
        try:
            db_instance = db
        except NameError:
            raise ValueError("Firebase must be initialized first. Run the Firebase initialization cell.")
    
    uploaded_events = []
    skipped_events = []
    event_judge_count = defaultdict(int)
    
    for event_name in event_list:
        try:
            # Get evaluation criteria
            eval_criteria = get_event_eval_criteria(event_name)
            
            # Get groups for this event
            groups = get_event_groups(event_name)
            
            # Get event code
            j_ref_id = get_event_code(event_name)
            
            # Create event document
            event_doc = {
                "name": event_name,
                "jRefId": j_ref_id,
                "group": groups,
                "evalCriteria": eval_criteria,
            }
            
            # Initialize judge lists if creating judges
            if create_judges:
                event_doc["judgeIdList"] = []
                event_doc["judgeEmailList"] = []
            
            # Upload to Firestore
            db_instance.collection("eventData_v2").document(event_name).set(event_doc)
            uploaded_events.append(event_name)
            
            # Create judges for this event
            if create_judges:
                for i in range(1, num_judges_per_event + 1):
                    judge_name = f"judge{i}{j_ref_id}"
                    judge_uid = addJudge(judge_name, event_name, db_instance)
                    if judge_uid:
                        event_judge_count[event_name] += 1
            
            print(f"✓ Uploaded: {event_name} (Groups: {groups}, Code: {j_ref_id}, Judges: {event_judge_count[event_name]})")
            
        except Exception as e:
            print(f"✗ Error uploading {event_name}: {str(e)}")
            skipped_events.append(event_name)
    
    print(f"\n✅ Successfully uploaded {len(uploaded_events)} events")
    if create_judges:
        total_judges = sum(event_judge_count.values())
        print(f"✅ Created {total_judges} judge accounts")
    if skipped_events:
        print(f"⚠ Skipped {len(skipped_events)} events due to errors")
    
    return uploaded_events, skipped_events, event_judge_count


# Optional: Helper function to delete existing judges (similar to 2024 cleanup)
def delete_existing_judges(db_instance=None, auth_instance=None):
    """
    Deletes all existing judge accounts from Firebase Auth and userData collection.
    Useful for cleanup before creating new judges.
    
    Args:
        db_instance: Firestore client instance (if None, will use global db)
        auth_instance: Firebase Auth instance (if None, will use global auth)
    """
    if db_instance is None:
        try:
            db_instance = db
        except NameError:
            raise ValueError("Firebase must be initialized first.")
    
    if auth_instance is None:
        auth_instance = auth
    
    print("Deleting all judges from Firebase Auth...")
    deleted_auth = 0
    for user in auth_instance.list_users().iterate_all():
        if "judge" in user.email.lower():
            try:
                auth_instance.delete_user(user.uid)
                deleted_auth += 1
            except Exception as e:
                print(f"  ⚠ Error deleting auth user {user.email}: {str(e)}")
    
    print(f"Deleted {deleted_auth} judge accounts from Firebase Auth")
    
    print("Deleting all judges from 'userData' collection...")
    deleted_userdata = 0
    users = db_instance.collection("userData").stream()
    for user in users:
        user_data = user.to_dict()
        if "judge" in user_data.get("email", "").lower() or user_data.get("role") == "judge":
            try:
                user.reference.delete()
                deleted_userdata += 1
            except Exception as e:
                print(f"  ⚠ Error deleting userData {user_data.get('email')}: {str(e)}")
    
    print(f"Deleted {deleted_userdata} judge documents from 'userData' collection")
    print("✅ Judge cleanup complete!")


print("Event upload functions defined successfully!")
print("\nNote: Make sure Firebase is initialized before uploading events.")
print("Run: cred = credentials.Certificate('path/to/credentials.json')")
print("     firebase_admin.initialize_app(cred)")
print("     db = firestore.client()")


In [ ]:
# Preview event metadata before uploading
# This shows what will be uploaded to Firestore

eventList = []

for rec in prepared_records_2025:
    eventList.append(rec.get("registeredEvents", ""))
eventList = [item for sublist in eventList for item in sublist]

eventList = [e for e in eventList if str(e).strip() != ""]
# drop 'No Second Competition' and ''
eventList = [e for e in eventList if str(e).strip() != "No Second Competition" and str(e).strip() != ""]

eventList = list(set(eventList))
eventList.sort()

eventList

print("="*80)
print("EVENT METADATA PREVIEW")
print("="*80)
print(f"\nTotal events: {len(eventList)}\n")

for event in eventList:
    criteria = get_event_eval_criteria(event)
    groups = get_event_groups(event)
    code = get_event_code(event)
    total_marks = sum(criteria.values())
    
    print(f"Event: {event}")
    print(f"  Code: {code}")
    print(f"  Groups: {groups}")
    print(f"  Total Marks: {total_marks}")
    print(f"  Criteria:")
    for criterion, marks in criteria.items():
        print(f"    - {criterion}: {marks} marks")
    print()

print("="*80)
print("Preview complete. Review above before uploading to Firestore.")
print("="*80)


In [ ]:
# Upload event metadata to Firestore
# This uses the dynamically generated eventList from the registration data

# Expand GROUP events to include Boys/Girls variants if needed
# According to PDF, GROUP events should be judged separately for Boys and Girls
# expanded_event_list = expand_group_events(eventList)
# expanded_event_list.sort()

print(f"Original event list: {len(eventList)} events")

# Upload events to Firestore with judge creation
# Set create_judges=False if you only want to upload events without creating judges
print("\n" + "="*60)
print("Uploading event metadata to Firestore...")
print("Creating judge accounts for each event...")
print("="*60 + "\n")

uploaded, skipped, event_judge_count = upload_event_metadata_to_firestore(
    eventList, 
    create_judges=True,  # Set to False to skip judge creation
    num_judges_per_event=4  # Number of judges per event (default: 4)
)

print("\n" + "="*60)
print("Upload Summary:")
print("="*60)
print(f"Total events processed: {len(eventList)}")
print(f"Successfully uploaded: {len(uploaded)}")
if skipped:
    print(f"Skipped/Errors: {len(skipped)}")
    print(f"Skipped events: {skipped}")
if event_judge_count:
    total_judges = sum(event_judge_count.values())
    print(f"\nTotal judges created: {total_judges}")
    print(f"Average judges per event: {total_judges / len(uploaded):.1f}")


In [ ]:
# plot numMaleAccompanyingNeedAccommodation

# prepared_df_2025['numMaleAccompanyingNeedAccommodation'].value_counts().plot(kind='bar')

# prepared_df_2025['numFemaleAccompanyingNeedAccommodation'].value_counts().plot(kind='bar')

# prepared_df_2025['numBoyAccompanyingKid'].value_counts().plot(kind='bar')

# prepared_df_2025['numGirlAccompanyingKid'].value_counts().plot(kind='bar')